## Especialización en Sistemas de Información Geográfica  
Laboratorio SIG  
Taller 03 - IMPACTO PARQUE NACIONAL POR EL USO DE LEÑA  
RAFAEL FABIAN SANCHEZ OSORIO

### 0. INTRODUCCIÓN

En la zona se desarrollan actividades agrícolas intensas de las cuales se obtiene leña para cocinar, 
desafortunadamente la leña obtenida de la agricultura no es suficiente para satisfacer la demanda de 
los habitantes de la zona. La única fuente de leña es el parque nacional donde no está permitida la 
recolección de leña (tala de vegetación leñosa), sin embargo, la gente ingresa ilegalmente al parque 
para abastecerse de leña.  
Para poder definir un manejo que reduzca el impacto en el parque nacional, se tiene que determinar las 
áreas principales con déficit de rendimiento de leña. Últimamente se está notando un cambio en la 
agricultura que agrava los problemas. Muchas de las zonas con plantaciones de café y cítricos (de alto 
rendimiento de leña) se están cambiando a cultivos como los frutales y hierbas aromáticas, con lo cual 
se logra una mayor ganancia económica, pero un menor rendimiento de leña, causando una presión 
más alta al parque nacional.

### 1 Insumos Modelo

In [1]:
import arcpy
from arcgis.gis import GIS
import os
import pandas as pd

gis = GIS()
arcpy.env.overwriteOutput = True

Insumos

In [2]:
geodatabaseInsumos = r'C:\Users\rfabi\Downloads\DATOS_T3.gdb'
veredas = os.path.join(geodatabaseInsumos,'VEREDAS_2021')
usos = os.path.join(geodatabaseInsumos,'USOS_2021')
poblados = os.path.join(geodatabaseInsumos,'POBLADOS_2021')
DTM10 = os.path.join(geodatabaseInsumos,'DEM')
vias = os.path.join(geodatabaseInsumos,'VIAS_2021')
tablaPoblacion = os.path.join(geodatabaseInsumos,'TAB_POBLACION')
tablaRendimiento = os.path.join(geodatabaseInsumos,'TAB_RENDIMIENTO')
tablaVelocidadVias = os.path.join(geodatabaseInsumos,'TAB_VELOCIDAD_VIAS')
tablaVelocidadUsos = os.path.join(geodatabaseInsumos,'TAB_VELOCIDAD_USOS')

Salidas

In [3]:

GDBResultados = r'C:\Users\rfabi\Downloads\DATOS_T3.gdb'
usosVeredas = os.path.join(GDBResultados,'UsosVeredas')
usos = os.path.join(geodatabaseInsumos,'USOS_2021')
poblados = os.path.join(geodatabaseInsumos,'POBLADOS_2021')
DTM10 = os.path.join(geodatabaseInsumos,'DEM')
vias = os.path.join(geodatabaseInsumos,'VIAS_2021')
tablaPoblacion = os.path.join(geodatabaseInsumos,'TAB_POBLACION')
tablaRendimiento = os.path.join(geodatabaseInsumos,'TAB_RENDIMIENTO')
tablaVelocidadVias = os.path.join(geodatabaseInsumos,'TAB_VELOCIDAD_VIAS')
tablaVelocidadUsos = os.path.join(geodatabaseInsumos,'TAB_VELOCIDAD_USOS')


veredasDemandaLena = os.path.join(GDBResultados,'DemandaLena')
pobladosDemanda = os.path.join(GDBResultados,'PobladosDemanda')
pobladosDeficit = os.path.join(GDBResultados,'PobladosDeficit')
parqueNacional = os.path.join(GDBResultados,'ParqueNacional')
parqueNacionalTiempos = os.path.join(GDBResultados,'ParqueNacionalTiempos')
accesibilidadPNN = os.path.join(GDBResultados,'AccesibilidadPNN')
accesibilidadPNNRaster = os.path.join(GDBResultados,'AccesibilidadPNNRaster')


pendienteRaster = os.path.join(GDBResultados,'PendienteRaster')
PendienteReclassRaster = os.path.join(GDBResultados, 'PendientesReclasficadas')
PendientesCorregidas = os.path.join(GDBResultados,'PendienteCorregidas')
ViasRaster = os.path.join(GDBResultados,'ViasRaster')
UsosRaster = os.path.join(GDBResultados,'UsosRaster')
VelocidadesRaster = os.path.join(GDBResultados,'Velocidades')
VelocidadesAjustadasRaster = os.path.join(GDBResultados,'VelocidadesAjustadas')
PesosRaster = os.path.join(GDBResultados,'Pesos')
TiemposData = os.path.join(GDBResultados,'Tiempos')

### 2.1 Disponibilidad de Leña
Para obtener la disponibilidad de leña se necesita la capa de uso y la información sobre el rendimiento 
de leña por cada tipo de uso por hectárea por año. También se tienen que utilizar el dato de consumo 
per cápita y la información de población disponible a nivel de unidad administrativa. Este análisis se 
realiza a nivel de unidad administrativa.



In [4]:
arcpy.analysis.TabulateIntersection(veredas, 'VEREDA', usos, usosVeredas, 'CLASES_USO')
veredasUSOData = arcpy.da.TableToNumPyArray(usosVeredas,('VEREDA','CLASES_USO','AREA'))
tablaRendimientoData = arcpy.da.TableToNumPyArray(tablaRendimiento,('CLASES_USO','RENDI'))
tablaPoblacionData = arcpy.da.TableToNumPyArray(tablaPoblacion,('VEREDA','POP_05','POP_01','POP_10','POB_2021'))

DFVeredadUSO = pd.DataFrame(veredasUSOData).infer_objects()
DFRendimientos = pd.DataFrame(tablaRendimientoData).infer_objects()
DFPoblacion = pd.DataFrame(tablaPoblacionData).infer_objects()

DFVeredadUSO['AREA'] = DFVeredadUSO['AREA'].apply(lambda x: round(x/10000,5))

Disponibilidad de Leña

In [5]:

DFVeredasRendimiento = pd.merge(DFVeredadUSO, DFRendimientos, on = 'CLASES_USO', how='left')
DFVeredasRendimiento['Rendimiento'] = DFVeredasRendimiento['AREA']*DFVeredasRendimiento['RENDI']
DFVeredasRendimiento = DFVeredasRendimiento.groupby('VEREDA').sum().reset_index()
DFVeredasRendimiento = DFVeredasRendimiento.drop('RENDI', axis=1)
DFVeredasRendimiento.head(10)

,VEREDA,AREA,Rendimiento
0,EL CAJON,7568.59245,991.861174
1,EL HATO,7101.11925,8175.574862
2,HATO VIEJO,2541.53438,486.704160
3,OJO DE AGUA,5609.81089,1078.539244
4,PALMERAS,9534.18300,8926.696287
5,PARAMO BAJO,10302.51710,2102.927656
6,PNN,1805.54568,0.000000
7,RASGATA ALTO,8938.87785,10468.575971
8,SALINAS,8569.16140,1770.915830


In [6]:
mapaRendimientoLeña = gis.map(location='Chiscas Boyacá')
arcpy.management.AddField(veredas, 'Rendimiento', 'FLOAT')
with arcpy.da.UpdateCursor(veredas, ['VEREDA','Rendimiento']) as cursor:
    for row in cursor:
        row[1] = DFVeredasRendimiento.loc[DFVeredasRendimiento['VEREDA'] == row[0]]['Rendimiento'].values[0]
        cursor.updateRow(row)
datosRendimiento = pd.DataFrame.spatial.from_featureclass(veredas)
datosRendimiento.spatial.plot(map_widget = mapaRendimientoLeña, col='Rendimiento', class_count=10,renderer_type='c',line_width=0.2)
mapaRendimientoLeña.legend = True
mapaRendimientoLeña

MapView(layout=Layout(height='400px', width='100%'), legend=True)

### 2.2 Demanda de Leña

In [7]:
DFVeredasRendimiento = pd.merge(DFVeredasRendimiento, DFPoblacion, on = 'VEREDA', how='left')
DFVeredasRendimiento['Consumo'] = DFVeredasRendimiento['POB_2021']*0.5*0.97
DFVeredasRendimiento['Demanda'] = DFVeredasRendimiento['Consumo']-DFVeredasRendimiento['Rendimiento']

arcpy.management.AddField(veredas, 'Consumo', 'FLOAT')
arcpy.management.AddField(veredas, 'Demanda', 'FLOAT')
with arcpy.da.UpdateCursor(veredas, ['VEREDA','Consumo','Demanda']) as cursor:
    for row in cursor:
        row[1] = DFVeredasRendimiento.loc[DFVeredasRendimiento['VEREDA'] == row[0]]['Consumo'].values[0]
        row[2] = DFVeredasRendimiento.loc[DFVeredasRendimiento['VEREDA'] == row[0]]['Demanda'].values[0]
        cursor.updateRow(row)
mapaDemandaLeña = gis.map(location='Chiscas Boyacá')
datosRendimiento = pd.DataFrame.spatial.from_featureclass(veredas)
DFVeredasRendimiento.head(10)

,VEREDA,AREA,Rendimiento,POP_05,POP_01,POP_10,POB_2021,Consumo,Demanda
0,EL CAJON,7568.59245,991.861174,553,162,1493,6563,3183.055,2191.193826
1,EL HATO,7101.11925,8175.574862,2127,431,2877,10220,4956.700,-3218.874862
2,HATO VIEJO,2541.53438,486.704160,681,389,1222,2621,1271.185,784.480840
3,OJO DE AGUA,5609.81089,1078.539244,696,402,1333,2964,1437.540,359.000755
4,PALMERAS,9534.18300,8926.696287,784,285,1553,4809,2332.365,-6594.331287
5,PARAMO BAJO,10302.51710,2102.927656,973,678,1617,6124,2970.140,867.212344
6,PNN,1805.54568,0.000000,0,0,0,0,0.000,0.000000
7,RASGATA ALTO,8938.87785,10468.575971,610,520,1049,2677,1298.345,-9170.230971
8,SALINAS,8569.16140,1770.915830,635,346,1273,4300,2085.500,314.584170


In [8]:
arcpy.analysis.Select(veredas, veredasDemandaLena, 'Demanda > 0')
datosDemanda = pd.DataFrame.spatial.from_featureclass(veredasDemandaLena)
datosDemanda.spatial.plot(map_widget = mapaDemandaLeña, col='Demanda', class_count=10,renderer_type='c',line_width=0.2)
mapaDemandaLeña.legend = True
mapaDemandaLeña

MapView(layout=Layout(height='400px', width='100%'), legend=True)

### 2.3 Accesibilidad a la Leña

2.3.1 Generando raster con los valores de corrección en velocidad por pendiente

In [9]:
pendiente = arcpy.sa.Slope(DTM10,"DEGREE")
pendiente.save(pendienteRaster)

pendienteReclasificadas = arcpy.sa.Reclassify(pendienteRaster, "Value",
    arcpy.sa.RemapRange(
        [[0,5,1],
        [5,10,2],
        [10,20,3],
        [20,30,4],
        [30,45,5],
        [45,65,6],
        [65,800,7]]))
pendienteReclasificadas.save(PendienteReclassRaster)

arcpy.management.AddField(PendienteReclassRaster, 'Correccion','DOUBLE')

correccionPendiente = {
    1 : 1,
    2 : 0.958,
    3 : 0.818,
    4 : 0.646,
    5 : 0.5,
    6 : 0.409,
    7 : 0.288
}

with arcpy.da.UpdateCursor(PendienteReclassRaster, ['Correccion', 'Value']) as cursor:
    for row in cursor:
        row[0] = correccionPendiente[row[1]]
        cursor.updateRow(row)

CorreccionPendienteData = arcpy.sa.Lookup(PendienteReclassRaster,'Correccion')
CorreccionPendienteData.save(PendientesCorregidas)

2.3.2 Generando Raster con las velocidades asignadas por clase de uso y vías

In [10]:
arcpy.conversion.FeatureToRaster(vias, 'VELOCIDAD', ViasRaster,12.5)
arcpy.conversion.FeatureToRaster(usos, 'VELOCIDAD', UsosRaster,12.5)
Velocidades = arcpy.sa.RasterCalculator([ViasRaster, UsosRaster],['x','y'],'Con(IsNull(x) ,y, x)')
Velocidades.save(VelocidadesRaster)
velocidadesAjustadas = arcpy.sa.Times(PendientesCorregidas,Velocidades)
velocidadesAjustadas.save(VelocidadesAjustadasRaster)
Pesos = arcpy.sa.RasterCalculator([VelocidadesAjustadasRaster],['x'],'Con(x==0,0.00001,3.6/x)')
Pesos.save(PesosRaster)

2.3.3 Extrayendo poblados que se encuentran dentro de las veredas con demanda

In [11]:
arcpy.analysis.Select(poblados, pobladosDemanda, "POBLADO NOT LIKE '%X'")
arcpy.analysis.Clip(pobladosDemanda, veredasDemandaLena, pobladosDeficit)
tiempos = arcpy.sa.CostDistance(pobladosDeficit,Pesos)
tiempos.save(TiemposData)

2.3.4 Extrayendo Parque Nacional

In [12]:
arcpy.analysis.Select(usos, parqueNacional, "CLASES_USO = 'PNN'")
parqueNacionalTime = arcpy.sa.ExtractByMask(TiemposData, parqueNacional)
parqueNacionalTime.save(parqueNacionalTiempos)
accesibilidadPNNRasterData = arcpy.sa.RasterCalculator([parqueNacionalTime],['x'],'Int(x)')
accesibilidadPNNRasterData.save(accesibilidadPNNRaster)

2.3.5 Estimando tiempos de Accesibilidad al Parque Nacional

In [13]:

arcpy.management.AddField(accesibilidadPNNRaster, 'AreaHa', 'DOUBLE')
arcpy.management.AddField(accesibilidadPNNRaster, 'AreaAcumulada', 'DOUBLE')
arcpy.management.AddField(accesibilidadPNNRaster, 'Impacto', 'TEXT')

def ImpactoArea(Area):
    if Area < 100:
        return '00 IMPACTO ALTO'
    elif (Area > 100 and Area<200):
        return '01 IMPACTO MEDIO'
    elif (Area>200 and Area<300):
        return '02 IMPACTO BAJO'
    else:
        return '03 SIN IMPACTO'  

with arcpy.da.UpdateCursor(accesibilidadPNNRaster, ['Count', 'AreaHa','AreaAcumulada']) as cursor:
    for row in cursor:
        area = 0
        row[1] = row[0]*12.5*12.5/10000
        area = area + row[1]
        row[2] = area
        cursor.updateRow(row)

with arcpy.da.UpdateCursor(accesibilidadPNNRaster, ['AreaHa','AreaAcumulada']) as cursor:
    area = 0
    for row in cursor:
        area = area+row[0]
        row[1] = area
        cursor.updateRow(row)
with arcpy.da.UpdateCursor(accesibilidadPNNRaster, ['AreaAcumulada','Impacto']) as cursor:
    for row in cursor:
        row[1] = ImpactoArea(row[0])
        cursor.updateRow(row)

arcpy.conversion.RasterToPolygon(accesibilidadPNNRaster,accesibilidadPNN,'','Impacto')

<Result 'C:\\Users\\rfabi\\Downloads\\DATOS_T3.gdb\\AccesibilidadPNN'>

In [14]:

mapaAccesibilidadPNN = gis.map(location='Chiscas Boyacá')
zonasImpactoPNN = pd.DataFrame.spatial.from_featureclass(accesibilidadPNN)
zonasImpactoPNN.spatial.plot(map_widget = mapaAccesibilidadPNN, col='Impacto', class_count=10,renderer_type='u',line_width=0.2)
mapaAccesibilidadPNN.legend = True
mapaAccesibilidadPNN

MapView(layout=Layout(height='400px', width='100%'), legend=True)